<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW4_%E6%96%87%E5%AD%97%E8%B3%87%E6%96%99%E5%B0%8F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 PTT 熱詞分析與摘要系統 (Gradio 介面)

*歡迎使用本系統！這是一個專為分析 PTT 日劇版 (Japandrama) 輿情而設計的自動化工具。*

*您可以透過這個網頁介面，一鍵啟動爬蟲抓取最新文章、分析熱門關鍵字，並利用 Gemini 自動生成專業的趨勢摘要。*

Google Sheet 連結：https://docs.google.com/spreadsheets/d/1mw8T-_jsLFGHZcq1EpreAZFjdY4TXoH-uPOtGyBKQW4/edit?usp=sharing

---

## 🚀 自動化流程執行 (主要功能)

這是本系統的核心功能頁籤。

### **1. 參數輸入**

在啟動流程前，您需要設定兩個參數：

* **要爬取的頁數**：設定要從 PTT 日劇版抓取多少頁的文章列表 (例如輸入 10，代表抓取最新的 10 頁)。
* **要統計的 Top N 熱詞數量**：設定您希望系統分析出前幾名的熱門關鍵字 (例如輸入 20，代表找出 Top 20 熱詞)。
* **🚀 一鍵啟動自動化流程**：設定好參數後，點擊此按鈕即可開始執行。系統會自動完成所有分析步驟。

### **2. 分析結果 (子分頁)**

分析開始後，您可以在下方的三個子分頁中查看即時進度與最終結果：

#### **🛠️ 技術日誌與輸出細節**
*這個分頁用來追蹤系統目前的執行狀態。*
* **詳細流程日誌**：系統會即時更新目前的進度，如果流程不幸發生錯誤，詳細的錯誤訊息也會顯示在此處。

#### **🕸️ 爬取之網站**
*這個分頁用來檢視原始資料。*
* **資料來源**：顯示目前爬取的看板 (例如：PTT 日劇版)。
* **網站連結**：點擊表格中的文章，可在上方顯示標題與連結，並於新分頁開啟原始 PTT 文章。
* **爬取文章列表 (原始資料)**：以表格形式顯示系統從 PTT 抓取到的所有文章標題、作者、日期與連結。

#### **✅ 最終結果**
*這是您最需要關注的分頁。* 這裡會顯示分析的最終產出：
* **🤖 Gemini 洞察摘要與結論**：由 AI 根據熱詞生成的專業分析報告，包含趨勢洞察與結論。
* **📈 Top N 熱詞視覺化圖表**：將前 N 名的熱門關鍵字以水平長條圖顯示，讓您一眼看出哪些詞彙最熱門。
* **📈 Top N 熱詞統計結果 (Data)**：顯示圖表的原始數據，包含關鍵字及其 TF-IDF 權重分數。

---

## 🔬 Jieba 分詞模式測試 (輔助工具)

這是一個獨立的小工具，用來測試中文分詞的效果。

1.  **輸入中文句子**：在輸入框中貼上您想測試的任何中文句子。
2.  **🔍 執行分析**：點擊按鈕。
3.  **查看結果**：下方會立即顯示四種不同的分詞模式結果，幫助您了解不同演算法是如何切分詞彙的。

---

## ☁️ 雲端自動備份 (後台功能)

除了在介面上顯示結果外，當您按下「一鍵啟動」時，系統也會在背景執行：

1.  將「爬取文章列表 (原始資料)」完整備份到指定的 **Google Sheets** 工作表 ("PTT文章列表")。
2.  將「Top N 熱詞統計結果」備份到另一張 **Google Sheets** 工作表 ("熱詞統計")。


In [ ]:
!pip install gspread google-generativeai gradio requests beautifulsoup4 jieba pandas scikit-learn

In [ ]:
# ================================
# 1. Google 授權
# ================================
from google.colab import auth
auth.authenticate_user()

# ================================
# 2. 檢查 Gemini API 金鑰
# ================================
from google.colab import userdata
try:
    api_key = userdata.get("gemini")
except Exception as e:
    print(f"🚨 讀取金鑰時發生錯誤：{e}")

In [ ]:
import gspread
import pandas as pd
import datetime
import pytz
import gradio as gr
import requests
from bs4 import BeautifulSoup
import jieba  # 使用針對繁體中文優化的 jieba
import jieba.analyse
import jieba.posseg as pseg
import google.generativeai as genai
from google.colab import auth, userdata
from google.auth import default
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import traceback

有改模型

In [ ]:
# Google Sheets 與 Gemini 授權與設定
try:
    # --- Google Sheets ---
    creds, _ = default()
    gc = gspread.authorize(creds)

    SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1mw8T-_jsLFGHZcq1EpreAZFjdY4TXoH-uPOtGyBKQW4/edit?usp=sharing"
    gsheets = gc.open_by_url(SPREADSHEET_URL)

    # --- Gemini API ---
    GEMINI_API_KEY = userdata.get("gemini")
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel("gemini-2.5-flash")

except Exception as e:
    print(f"🚨 授權或設定時發生錯誤：{e}")

In [ ]:
# --- 3.1 PTT 爬蟲 (Helper 函式) ---
def get_previous_page_url(soup):
    """獲取上一頁 (更舊的文章) 的連結"""
    paging_div = soup.find('div', class_='btn-group btn-group-paging')
    if paging_div:
        prev_button = paging_div.find_all('a')[1]
        if 'href' in prev_button.attrs:
            return "https://www.ptt.cc" + prev_button['href']
    return None

def extract_index_from_url(url):
    """從 PTT 網址中解析出頁碼"""
    try:
        # 使用正規表達式匹配 index 後面的數字
        match = re.search(r'index(\d+)\.html', url)
        if match:
             return int(match.group(1))
        return None
    except Exception:
        return None

# --- 3.1 PTT 爬蟲 (主要函式) ---
def scrape_ptt_japandrama(pages_to_fetch, log_output):
    """爬取 PTT Japandrama 版指定頁數的文章列表，並記錄詳細日誌"""
    BOARD_NAME = "Japandrama"
    START_URL = f"https://www.ptt.cc/bbs/{BOARD_NAME}/index.html"
    BASE_URL = f"https://www.ptt.cc/bbs/{BOARD_NAME}/index"

    session = requests.Session()
    session.post("https://www.ptt.cc/ask/over18", data={"yes": "yes"})

    all_data_list = []
    log_output.append(f"--- 1. 爬蟲日誌 ---")
    log_output.append(f"目標看板: {BOARD_NAME} | 爬取頁數: {pages_to_fetch}")

    try:
        # 1. 取得最新的頁碼
        r = session.get(START_URL, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        prev_page_url = get_previous_page_url(soup)

        if not prev_page_url:
            log_output.append("❌ 錯誤：找不到 '上一頁' 連結，無法定位最新頁碼。")
            return pd.DataFrame(), log_output

        start_index = extract_index_from_url(prev_page_url)
        if start_index is None:
            log_output.append("❌ 錯誤：無法從連結中解析出起始頁碼。")
            return pd.DataFrame(), log_output

        log_output.append(f"起始頁碼 (前一頁): {start_index}。將從 {start_index} 遞減爬取。")

        # 先爬取最新一頁
        log_output.append(f" -> 爬取頁面 (最新): {START_URL}")
        articles = soup.find_all("div", class_="r-ent")
        for art in articles:
          title_tag = art.find("div", class_="title").find("a")
          if title_tag and "[公告]" not in title_tag.text:
            title = title_tag.text
            link = "https://www.ptt.cc" + title_tag["href"]
            author = art.find("div", class_="author").text
            date_str = art.find("div", class_='date').text.strip()
            all_data_list.append({
              "日期": date_str, "作者": author, "標題": title, "連結": link
            })

        time.sleep(0.1)

        # 2. 迴圈爬取
        pages_remaining = pages_to_fetch - 1
        if pages_remaining > 0:
          stop_index = start_index - pages_remaining
          for index in range(start_index, stop_index, -1):
              url = f"{BASE_URL}{index}.html"
              log_output.append(f" -> 爬取頁面 {index}:{url}")

              try:
                  r_page = session.get(url, timeout=10)
                  if r_page.status_code != 200:
                      log_output.append(f"   ⚠️ 跳過頁面 (狀態碼: {r_page.status_code})")
                      continue

                  soup_page = BeautifulSoup(r_page.text, "html.parser")
                  articles = soup_page.find_all("div", class_="r-ent")

                  for art in articles:
                      title_tag = art.find("div", class_="title").find("a")
                      if title_tag and "[公告]" not in title_tag.text:
                          title = title_tag.text
                          link = "https://www.ptt.cc" + title_tag["href"]
                          author = art.find("div", class_="author").text
                          date_str = art.find("div", class_='date').text.strip()
                          all_data_list.append({
                              "日期": date_str, "作者": author, "標題": title, "連結": link
                          })

                  time.sleep(0.1) # 縮短延遲以加速示範

              except requests.exceptions.RequestException as e:
                  log_output.append(f"   ❌ 爬取失敗: {e}")

    except requests.exceptions.RequestException as e:
        log_output.append(f"❌ 爬蟲起始請求失敗：{e}")
        return pd.DataFrame(), log_output

    df = pd.DataFrame(all_data_list)
    log_output.append(f"✅ 爬蟲結束。共抓取 {len(df)} 篇文章。")
    return df, log_output


# --- 3.2 讀寫 Google Sheet ---
def get_or_create_worksheet(sheet, title):
    try:
        worksheet = sheet.worksheet(title)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=title, rows="100", cols="20")
    return worksheet

def write_to_sheet(sheet, worksheet_name, df, log_output):
    log_output.append(f"--- 2. Google Sheet 寫入日誌 ---")
    try:
        worksheet = get_or_create_worksheet(sheet, worksheet_name)
        worksheet.clear()
        worksheet.update(
            [df.columns.values.tolist()] + df.astype(str).values.tolist(),
            value_input_option="USER_ENTERED"
        )
        log_output.append(f"✅ 成功寫入 {worksheet_name} 工作表 ({len(df)} 筆資料)。")
    except Exception as e:
        log_output.append(f"❌ 寫入 Sheet 失敗: {e}")
    return log_output

# --- 3.3 TF-IDF 關鍵字分析 ---

STOPWORDS = set(['的', '了', '是', '在', '我', '你', '他', '她', '之', '一個', '和',
                 '討論', '分享', '心得', '問題', '請益', '情報', 'LIVE', 'Re', 're', 'EP', 'OST', 'EP', 'ep', '2025',
                 '日劇', '日', '劇', '電影', '影']) # 增加與看板相關的詞

def get_tfidf_keywords(df, top_n, log_output):
    """使用 sklearn.TfidfVectorizer 進行 TF-IDF 分析並記錄詳細日誌"""

    log_output.append(f"--- 3. TF-IDF 分析日誌 (Sklearn) ---")
    log_output.append(f"目標關鍵字數量: Top {top_n}")
    log_output.append(f"停用詞數量: {len(STOPWORDS)}")

    if '標題' not in df.columns or df['標題'].dropna().empty:
        log_output.append("❌ 錯誤: 資料集中缺少 '標題' 欄位或標題為空。")
        return pd.DataFrame(), log_output

    document_list = []

    # 1. 文本預處理與分詞
    for title in df['標題'].dropna():
        cleaned_text = re.sub(r"\[.*?\]", "", title).strip()
        cleaned_text = re.sub(r'[^\w\s]', ' ', cleaned_text)
        cleaned_text = re.sub(r'\b(ep|re)\d+\b', ' ', cleaned_text, flags=re.IGNORECASE)

        # 使用精確模式分詞
        words = jieba.lcut(cleaned_text, cut_all=False)

        # 過濾停用詞和單字
        filtered_words = [
            word.strip()
            for word in words
            if word.strip() and len(word.strip()) > 1 and word.strip().lower() not in STOPWORDS
        ]

        if filtered_words:
            document_list.append(" ".join(filtered_words))

    log_output.append(f"已將 {len(df)} 篇標題分詞並過濾，產生 {len(document_list)} 篇有效文檔。")

    if not document_list:
        log_output.append("⚠️ 沒有可分析的文檔 (可能都被過濾了)。")
        return pd.DataFrame(), log_output

    # 2. TF-IDF 計算
    try:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(document_list)
        feature_names = vectorizer.get_feature_names_out()
        tfidf_array = tfidf_matrix.toarray()

        log_output.append(f"✅ TF-IDF 矩陣建立成功。詞彙總數 (Features): {len(feature_names)}")

        # 3. 計算平均權重 (找出整體重要性)
        avg_tfidf_scores = defaultdict(float)
        num_documents = len(document_list)

        for doc_weights in tfidf_array:
            for i, weight in enumerate(doc_weights):
                word = feature_names[i]
                avg_tfidf_scores[word] += weight

        for word in avg_tfidf_scores:
            avg_tfidf_scores[word] /= num_documents

        # 4. 排序並選出 Top N
        sorted_avg_tfidf = sorted(avg_tfidf_scores.items(), key=lambda item: item[1], reverse=True)
        top_keywords_df = pd.DataFrame(sorted_avg_tfidf[:top_n], columns=['關鍵字', 'TF-IDF平均權重'])

        log_output.append(f"✅ 成功提取 Top {len(top_keywords_df)} 個關鍵字。")
        log_output.append(f"Top 5 關鍵字範例: {', '.join([k[0] for k in sorted_avg_tfidf[:5]])}")

        return top_keywords_df, log_output

    except ValueError as e:
        log_output.append(f"❌ TF-IDF 分析失敗 (ValueError): {e}")
        return pd.DataFrame(), log_output
    except Exception as e:
        log_output.append(f"❌ TF-IDF 分析發生未預期錯誤: {e}")
        return pd.DataFrame(), log_output

# --- 3.4 Gemini API 生成摘要 (新增日誌) ---
def get_gemini_summary(keywords_df, log_output):
    """使用 Gemini API 根據關鍵字生成摘要"""

    log_output.append(f"--- 4. Gemini 摘要日誌 ---")
    if keywords_df.empty:
        log_output.append("⚠️ 缺少關鍵字，無法生成摘要。")
        return "⚠️ 沒有關鍵字，無法生成摘要。", log_output

    keywords_list = keywords_df['關鍵字'].tolist()
    prompt = f"""
    您是一位專業的數據分析師，專精於社群輿情分析。

    任務：
    請根據 PTT 日劇版 (JapanDrama) 的 {len(keywords_list)} 個熱門關鍵字，生成一份專業的分析報告。

    熱門關鍵字 (依 TF-IDF 平均權重排序)：
    {', '.join(keywords_list)}

    輸出格式要求 (請嚴格遵守)：
    1.  **五句洞察摘要**：條列式，每句都是精闢的觀察。
    2.  **一段 120 字結論**：總結目前的趨勢或現象。

    請使用繁體中文回答。
    """

    try:
        log_output.append(f"模型請求參數: gemini-2.5-flash, 關鍵字數量: {len(keywords_list)}")
        response = model.generate_content(prompt, request_options={"timeout": 120})
        clean_text = response.text.replace("#", "").replace("*", "")
        log_output.append("✅ 摘要生成成功。")
        return clean_text, log_output
    except Exception as e:
        log_output.append(f"❌ Gemini API 呼叫失敗：{e}")
        return f"❌ Gemini API 呼叫失敗：{e}", log_output

# --- 3.5 獨立的 Jieba 分詞展示函式 (新增) ---
def demonstrate_jieba_modes(sentence):
    """展示 Jieba 的四種分詞模式"""
    if not sentence:
        return "請輸入中文句子進行測試", "請輸入中文句子進行測試", "請輸入中文句子進行測試", "請輸入中文句子進行測試"

    output = f"原始句子: {sentence}\n\n"

    # 1. 精確模式 (Default Mode)
    seg_list_precise = jieba.cut(sentence, cut_all=False)
    precise_result = '/ '.join(seg_list_precise)

    # 2. 全模式 (Full Mode)
    seg_list_all = jieba.cut(sentence, cut_all=True)
    all_result = '/ '.join(seg_list_all)

    # 3. 搜尋引擎模式 (Search Engine Mode)
    seg_list_search = jieba.cut_for_search(sentence)
    search_result = '/ '.join(seg_list_search)

    # 4. 詞性標註
    words = pseg.cut(sentence)
    pos_result_list = [f"{word}/{flag}" for word, flag in words]
    pos_result = ' '.join(pos_result_list)

    return precise_result, all_result, search_result, pos_result


# --- 3.6 Gradio 輔助函式 (點擊表格) ---
def show_selected_link(data: pd.DataFrame, evt: gr.SelectData):
    """當使用者點擊 Dataframe 中的某一列時，在 Markdown 中顯示該列的連結"""
    if evt.index is None or not evt.selected:
        return "*點擊上方表格中的任一文章，連結將顯示於此*"
    try:
        selected_row_index = evt.index[0] # 獲取列索引
        if selected_row_index >= len(data):
             return "*表格資料已變動，請重新點擊*"

        selected_row = data.iloc[selected_row_index]
        title = selected_row.get('標題', '無標題')
        link = selected_row.get('連結', None)

        if link:
            # Markdown 連結 (Gradio 會自動使其在新分頁開啟)
            return f"### 點擊開啟連結：\n## [{title}]({link})"
        else:
            return f"**{title}**\n\n(此列沒有可用的 '連結' 欄位)"
    except Exception as e:
        return f"無法讀取連結：{e}"

In [ ]:
# ================================
# 4. Gradio 整合函式
# ================================
def run_full_automation_flow(top_n_str, pages_to_fetch_str):
    """Gradio 點擊後執行的完整流程"""

    empty_df = pd.DataFrame()
    empty_str = ""
    log_output = []

    site_list = ["PTT 日劇版 (Japandrama)"]
    # 清空上次結果
    yield "日誌將顯示於此...", empty_df, empty_str, None, gr.Radio(choices=["尚未執行"], value="尚未執行"), empty_df

    # --- 參數驗證 ---
    try:
        top_n = int(top_n_str)
        pages_to_fetch = int(pages_to_fetch_str)
        if top_n <= 0 or pages_to_fetch <= 0:
            log_output.append("❌ Top N 或爬取頁數必須是大於 0 的數字。")
            yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list), empty_df
            return
    except ValueError:
        log_output.append("❌ 請輸入有效的數字。")
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list), empty_df
        return

    # --- 自動化流程 ---
    log_output.append("===================================================")
    log_output.append(f"🚀 自動化流程啟動 ({datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")
    log_output.append("===================================================")

    try:
        # --- 步驟 1: 爬蟲 ---
        log_output.append("1/4: 🏃‍♂️ 開始爬取 PTT JapanDrama 版文章...")
        # [修正 Yield 1] 補上第 5 個值 empty_df
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=site_list[0]), empty_df

        scraped_df, log_output = scrape_ptt_japandrama(pages_to_fetch, log_output)
        # [修正 Yield 2] scraped_df 已更新
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=site_list[0]), scraped_df

        if scraped_df.empty:
            log_output.append("❌ 爬蟲失敗，未抓取到任何資料。流程終止。")
            yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list), empty_df
            return

        # --- 步驟 2: 寫入 Sheet (可選) ---
        log_output = write_to_sheet(gsheets, "PTT文章列表", scraped_df, log_output)

        # --- 步驟 3: TF-IDF 分析 ---
        log_output.append("2/4: 📊 正在進行 Sklearn TF-IDF 關鍵字分析...")
        # [修正 Yield 3] 補上第 5 個值 scraped_df (因為爬蟲列表應繼續顯示)
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=site_list[0]), scraped_df

        keywords_df, log_output = get_tfidf_keywords(scraped_df, top_n, log_output)

        plot_df = None # 先宣告
        if not keywords_df.empty:
          # 為了讓 BarPlot 頂部顯示權重最高的，我們需將 df 升冪排序
          plot_df = keywords_df.sort_values("TF-IDF平均權重", ascending=True)

        # [修正 Yield 4] 第 2 個值更新為 keywords_df
        yield "\n".join(log_output), keywords_df, empty_str, plot_df, gr.Radio(choices=site_list, value=site_list[0]), scraped_df

        if keywords_df.empty:
            log_output.append("⚠️ 分析完成，但未提取到關鍵字。流程終止。")
            # [修正 Yield 5] 第 2 個值是 empty_df (正確)，但第 5 個值 scraped_df 應保留
            yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list), scraped_df
            return

        # --- 步驟 4: 寫入 Sheet (可選) ---
        log_output.append("3/4: 📈 正在將 Top 熱詞回寫至 Sheet (熱詞統計)...")
        log_output = write_to_sheet(gsheets, "熱詞統計", keywords_df, log_output)

        # --- 步驟 5: Gemini 摘要 ---
        log_output.append("4/4: 🧠 正在呼叫 Gemini API 生成摘要...")
        # [修正 Yield 6] 第 2 個值是 keywords_df (應保留)，補上第 5 個值 scraped_df
        yield "\n".join(log_output), keywords_df, empty_str, plot_df, gr.Radio(choices=site_list, value=site_list[0]), scraped_df

        summary, log_output = get_gemini_summary(keywords_df, log_output)
      	# [修正 Yield 7] 第 2 個值是 keywords_df，第 3 個值更新為 summary
        yield "\n".join(log_output), keywords_df, summary, plot_df, gr.Radio(choices=site_list, value=site_list[0]), scraped_df

        log_output.append("===================================================")
        log_output.append("✅ 全部流程完成！請切換到「最終結果」標籤頁查看。")

        # 最終回傳 (您原本這行就是對的)
        yield "\n".join(log_output), keywords_df, summary, plot_df, gr.Radio(choices=site_list, value=site_list[0]), scraped_df

    except Exception as e:
        error_msg = f"❌ 流程發生未預期錯誤：{e}\n{traceback.format_exc()}"
        log_output.append(error_msg)
      	# [修正 Yield 8] 確保錯誤回傳也是 5 個值
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=site_list[0]), empty_df

In [ ]:
# ================================
# 5. 啟動 Gradio 介面 (已升級 Tab 佈局)
# ================================
print("\n🚀 正在啟動 Gradio 介面...")

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 📊 PTT 熱詞分析與摘要系統

        """
    )

    with gr.Tab("🚀 自動化流程執行"):
        with gr.Row():
            pages_to_fetch_input = gr.Textbox(label="要爬取的頁數", value="10", scale=1)
            top_n_input = gr.Textbox(label="要統計的 Top N 熱詞數量", value="20", scale=1)
            run_btn = gr.Button("🚀 一鍵啟動自動化流程", variant="primary", scale=2)

        gr.Markdown("---")

        # 使用 Tab 來區分最終結果和日誌
        with gr.Tabs():

            with gr.TabItem("🛠️ 技術日誌與輸出細節"):
                # 使用 Textbox 來顯示詳細的日誌
                log_output_text = gr.Textbox(
                    label="詳細流程日誌 (爬蟲、寫入、分析步驟)",
                    lines=30,
                    interactive=False,
                    show_copy_button=True
                )
            with gr.TabItem("🕸️ 爬取之網站"):
                site_list_output = gr.Radio(
                    label="資料來源",
                    choices=["尚未執行"],
                    value="尚未執行",
                    interactive=False
                )
                gr.Markdown("---")

                link_display_output = gr.Markdown(
          			value="*點擊上方表格中的任一文章，連結將顯示於此*"
          		  )

                scraped_data_output = gr.Dataframe(
              		label="爬取文章列表 (原始資料)",
              		headers=["日期", "作者", "標題", "連結"], # 預先定義欄位
              		interactive=True,
              		row_count=(15, 'dynamic')
              	)



            with gr.TabItem("✅ 最終結果"):
                summary_output = gr.Markdown(label="🤖 Gemini 洞察摘要與結論")

                keyword_plot_output = gr.BarPlot(
                  label="📈 Top N 熱詞視覺化圖表",
                  x="TF-IDF平均權重", # X 軸 (值)
                  y="關鍵字",       # Y 軸 (類別)
                  tooltip=['關鍵字', 'TF-IDF平均權重'],
                  color="TF-IDF平均權重",
                  vertical=False, # 設為水平長條圖
                  height=400
                )


                keywords_output = gr.Dataframe(label="📈 Top N 熱詞統計結果 (Sklearn TF-IDF 平均權重)")


        # 設定點擊事件
        run_btn.click(
          fn=run_full_automation_flow,
          	inputs=[top_n_input, pages_to_fetch_input],
          	outputs=[
                log_output_text,
                keywords_output,
                summary_output,      # 3. 摘要 (Markdown)
      		      keyword_plot_output,
                site_list_output,
                scraped_data_output
          	]
        )

        scraped_data_output.select(
      		fn=show_selected_link,
      		inputs=[scraped_data_output], # 將 Dataframe 本身作為輸入
      		outputs=[link_display_output], # 將連結輸出到 Markdown
      		show_progress=False # 點擊時不需要顯示 "loading"
    	  )

    with gr.Tab("🔬 Jieba 分詞模式測試"):
        gr.Markdown("### 中文分詞模式比較")
        jieba_input = gr.Textbox(
            label="輸入中文句子 (例如: 日本電視台與製作公司合作推出驚悚新劇)",
            value="日本電視台與製作公司合作推出驚悚新劇",
            lines=2
        )

        jieba_run_btn = gr.Button("🔍 執行分析")

        # 使用 Row 來水平展示四種模式
        with gr.Row():
            precise_output = gr.Textbox(label="1. 精確模式 (最常用)")
            all_output = gr.Textbox(label="2. 全模式 (所有可能組合)")
        with gr.Row():
            search_output = gr.Textbox(label="3. 搜尋引擎模式 (長詞再切)")
            pos_output = gr.Textbox(label="4. 詞性標註 (詞語/詞性)")

        jieba_run_btn.click(
            fn=demonstrate_jieba_modes,
            inputs=[jieba_input],
            outputs=[precise_output, all_output, search_output, pos_output],
        )


demo.queue().launch(share=True, debug=True)


🚀 正在啟動 Gradio 介面...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db54f8bc2c74d9575d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://db54f8bc2c74d9575d.gradio.live
